In [1]:
import os
cwd = os.getcwd()
print(cwd)

C:\Users\akk\Clubcorp\CommunityBenefit


In [2]:
import pandas as pd
import sqlite3

In [3]:
dining_benefit_df = pd.read_csv("output\intermediate\Community_Dining_Benefits.csv")

In [4]:
dining_benefit_df.head()

,community,benefit_package_id,benefit_id,benefit
0,5338,050e2a6d5c64a8e416673096d42bf16c,d56dc079ad791b356e44f96fdf899fb4,50% OFF TWO MEMBER MEALS PER CLUB
1,5338,050e2a6d5c64a8e416673096d42bf16c,120cd73056daa6f473aabd645401b4e9,LUNCH ACCESS
2,5334,94c705581c4b62602e9e13105ed33351,c6994c49a7a41151e993a4d84fee8e62,50% OFF TWO MEMBER MEALS PER CLUB
3,5334,94c705581c4b62602e9e13105ed33351,38262002de457cedc3354470b1e66286,LUNCH ACCESS
4,5322,f00b6c1a5e377bdf937d6a924113cd47,c66d5597e371e64ff903708cb173a386,ONE 2-4-1 PER CLUB DINING


In [5]:
dining_benefit_metadata_df = pd.read_csv("datasets\metadata\dining_benefit_meta_data.csv")

In [6]:
dining_benefit_metadata_df.head()

,benefit,UsageType,IsQuantityUnlimited,Quantity,Period,Scope,MaxPerClub
0,DINING ACCESS,Dining Access,Yes,NaN,Per Month,Per Club,NaN
1,PAY TO DINE,Dining Access,Yes,NaN,Per Month,Per Club,NaN
2,UNLIMITED 2-4-1,2-4-1,Yes,NaN,Per Month,Per Club,NaN
3,ONE 2-4-1 TOTAL,2-4-1,NaN,1.0,Per Month,Per Community,NaN
4,ONE 2-4-1 PER DINING OR GOLF,2-4-1 Dining or Golf,NaN,1.0,Per Month,Per Community,NaN


In [7]:
%%time
conn = sqlite3.connect("diningBenefit.db")
cur = conn.cursor()
dining_benefit_df.to_sql("diningBenefit", conn, if_exists="replace")
dining_benefit_metadata_df.to_sql("diningBenefitMetadata", conn, if_exists="replace")

Wall time: 108 ms


In [8]:
query = """select diningBenefit.community,
         diningBenefit.benefit_package_id,
         diningBenefit.benefit_id,
         diningBenefit.benefit,
         diningBenefitMetadata.UsageType,
         diningBenefitMetadata.IsQuantityUnlimited,
         diningBenefitMetadata.Quantity,
         diningBenefitMetadata.Period,
         diningBenefitMetadata.Scope,
         diningBenefitMetadata.MaxPerClub
         from diningBenefit
         left join diningBenefitMetadata on trim(upper(diningBenefitMetadata.benefit)) = trim(upper(diningBenefit.benefit))"""

rs_benefit = pd.read_sql_query(query, conn)

In [9]:
print("Total number of dining benefits: " + str(len(rs_benefit)))

Total number of dining benefits: 261


<b>Save Golf Benefits With Additional Columns<b>

In [10]:
rs_benefit.to_csv("output/Community_Dining_Benefit_Extended.csv",index=False)

In [11]:
conn.execute("DROP TABLE diningBenefit")
conn.execute("DROP TABLE diningBenefitMetadata")
cur.close()
conn.close()